# SNCF Prépa 

## Exo 1 - Connaissance des actifs 

In [2]:
import pandas as pd

actifs = pd.DataFrame({
    "asset_id": [1, 2, 3, 4, 5],
    "type_actif": ["Voie", "Signalisation", "Ouvrage", "Voie", "Signalisation"],
    "ligne": ["L1", "L1", "L2", "L3", "L3"],
    "annee_mise_service": [1980, 1995, 1970, 2005, 2010],
    "etat": ["Bon", "Moyen", "Mauvais", "Bon", None]
})

## 1. Identifier les champs critiques manquants 

In [17]:
for col in actifs : 
    print(f"enregistrement manquant pour la colonne '{col}'") if any(actifs[col].isna()) else None # Pour garder la logique sur 1 ligne sinon mettre continue

enregistrement manquant pour la colonne 'etat'


## 2. Âge des actifs

In [29]:
annee = 2026
for index, ligne in actifs.iterrows():
    age = annee - ligne["annee_mise_service"]
    print(f"{ligne['type_actif']} : {age} ans")

Voie : 46 ans
Signalisation : 31 ans
Ouvrage : 56 ans
Voie : 21 ans
Signalisation : 16 ans


## 3. Classer les actifs en risque faible / moyen / élevé 

In [38]:
actifs.sort_values(by="etat",ascending=False)

,asset_id,type_actif,ligne,annee_mise_service,etat
1,2,Signalisation,L1,1995,Moyen
2,3,Ouvrage,L2,1970,Mauvais
0,1,Voie,L1,1980,Bon
3,4,Voie,L3,2005,Bon
4,5,Signalisation,L3,2010,None


## 4. Proposer une règle de qualité des données 

In [40]:
# Vérifier que asset_id = unique
# Un actif très ancien ne peut pas être en Bon état sans justification 
# Etat forcément Bon - Moyen - Mauvais
# ...

## Exo 2 - Modélisation objet (vision AMOA)

In [41]:
interventions = pd.DataFrame({
    "intervention_id": [101, 102, 103, 104],
    "asset_id": [1, 2, 2, 5],
    "type_intervention": ["Inspection", "Maintenance", "Maintenance", "Inspection"],
    "date": ["2023-01-10", "2023-03-05", "2024-02-20", "2024-01-15"],
    "cout": [500, 2000, 1800, 400]
})


## 1. Identifier les objets métiers 
Objet métier = entité du monde réel que le métier manipule, suit, décide ou pilote <br>
Objet Métier -> Actif (un élément du patrimoine ferroviaire), Ligne (une infrastructure ferroviaire), Budget (une enveloppe financière)

Les objets métier identifiés sont : l’Actif, entité physique du patrimoine, et l’Intervention, qui représente une action réalisée sur un actif et porte les informations de coût et de date nécessaires au pilotage.

## Exo 3 - Pilotage de la qualité des données 

In [54]:
qualite = pd.DataFrame({
    "objet": ["Actif", "Actif", "Intervention", "Intervention"],
    "champ": ["etat", "annee_mise_service", "cout", "date"],
    "taux_completude": [0.92, 0.85, 0.99, 0.90]
})

## 1. Identifier les champs non conformes (< 95%)

In [55]:
for index, ligne in qualite.iterrows() : 
    print(f"{ligne['champ']} : {int(ligne['taux_completude'] * 100)} %") if ligne['taux_completude'] < 0.95 else None

etat : 92 %
annee_mise_service : 85 %
date : 90 %


## 2. Créer un indicateur global de qualité 

In [56]:
qualite["indicateur_qualite"] = "Mauvais"
qualite.loc[qualite["taux_completude"] >= 0.95, "indicateur_qualite"] = "Bon"
qualite

,objet,champ,taux_completude,indicateur_qualite
0,Actif,etat,0.92,Mauvais
1,Actif,annee_mise_service,0.85,Mauvais
2,Intervention,cout,0.99,Bon
3,Intervention,date,0.90,Mauvais


## Exo 4. Collecte & qualification des données 

In [75]:
projets = pd.DataFrame({
    "projet_id": [1, 2, 3, 4],
    "ligne": ["L1", "L2", "L3", "L1"],
    "budget_prevu": [10_000_000, 5_000_000, None, 8_000_000],
    "budget_reel": [9_500_000, None, 6_200_000, 8_300_000],
    "statut": ["Terminé", "En cours", "Terminé", "En cours"]
})


## 1. Identifier les incohérences budgétaire 

In [76]:
for index, ligne in projets.iterrows() : 
    if ligne['budget_prevu'] != ligne['budget_reel'] :
        print(f"Incohérence -> prévision :{ligne['budget_prevu']} - réalité : {ligne['budget_reel']}")
    else :
        continue
    

Incohérence -> prévision :10000000.0 - réalité : 9500000.0
Incohérence -> prévision :5000000.0 - réalité : nan
Incohérence -> prévision :nan - réalité : 6200000.0
Incohérence -> prévision :8000000.0 - réalité : 8300000.0


## 2. Compléter les valeurs manquantes (règles métier) 
Je vais inventer règle métier. Si budget_prevu inconnu alors 1 000 000, si budget_reel inconnu alors 1 500 000

In [77]:
projets.loc[projets["budget_prevu"].isna(), "budget_prevu"] = 1_000_000
projets.loc[projets["budget_reel"].isna(), "budget_reel"] = 1_500_000

In [78]:
projets

,projet_id,ligne,budget_prevu,budget_reel,statut
0,1,L1,10000000.0,9500000.0,Terminé
1,2,L2,5000000.0,1500000.0,En cours
2,3,L3,1000000.0,6200000.0,Terminé
3,4,L1,8000000.0,8300000.0,En cours


## 3. Indicateur écart budgétaire 

In [79]:
projets["ecart_budget"] = projets["budget_prevu"] - projets["budget_reel"]
projets 

,projet_id,ligne,budget_prevu,budget_reel,statut,ecart_budget
0,1,L1,10000000.0,9500000.0,Terminé,500000.0
1,2,L2,5000000.0,1500000.0,En cours,3500000.0
2,3,L3,1000000.0,6200000.0,Terminé,-5200000.0
3,4,L1,8000000.0,8300000.0,En cours,-300000.0


## 4. Filtrer les projets à risque (reel > théorique)

In [80]:
projets.loc[projets["ecart_budget"] <0, :]

,projet_id,ligne,budget_prevu,budget_reel,statut,ecart_budget
2,3,L3,1000000.0,6200000.0,Terminé,-5200000.0
3,4,L1,8000000.0,8300000.0,En cours,-300000.0


## Exercice 5 - Nettoyage et uniformisation

In [81]:
travaux = pd.DataFrame({
    "ligne": ["l1", "L1 ", "L2", "l3", "L3"],
    "type_travaux": ["Renouvellement", "renouvellement", "Maintenance", "maintenance", "Maintenance"],
    "cout": [100000, 110000, 50000, 52000, None]
})


## 1. Uniformiser les libellés 

In [116]:
for col in travaux.columns:
    if travaux[col].dtype == "object":  # seulement les colonnes texte
        # on rajoute str spécialement pour ce cas car Series 
        # strip -> supprimer les espaces
        travaux[col] = travaux[col].str.strip().str.capitalize() # capitalize permet de garder la 1er occurence en majuscule 
travaux

,ligne,type_travaux,cout
0,L1,Renouvellement,100000.0
1,L1,Renouvellement,110000.0
2,L2,Maintenance,50000.0
3,L3,Maintenance,52000.0
4,L3,Maintenance,NaN


## 2. Détecter les doublons fonctionnels 

In [119]:
travaux.drop_duplicates() # spécifier les colonnes si besoin 

,ligne,type_travaux,cout
0,L1,Renouvellement,100000.0
1,L1,Renouvellement,110000.0
2,L2,Maintenance,50000.0
3,L3,Maintenance,52000.0
4,L3,Maintenance,NaN


## 3. Corriger les valeurs manquantes 

In [126]:
travaux.loc[travaux["cout"].isna(),"cout"] = 50_000
travaux 

,ligne,type_travaux,cout
0,L1,Renouvellement,100000.0
1,L1,Renouvellement,110000.0
2,L2,Maintenance,50000.0
3,L3,Maintenance,52000.0
4,L3,Maintenance,50000.0


## EXO 6 — Analyse de performance des actifs